<a href="https://colab.research.google.com/github/Bprs68/Rossmann-Sales-Prediction-/blob/main/Optional_Model_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder


import warnings    
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path = '/content/drive/MyDrive/AlmaBetter/Capstone Projects/Capstone Project - Supervised Machine Learning- Regression/cleaned_data/'

In [4]:
train_df = pd.read_csv(file_path+'train.csv')

In [5]:
test_df = pd.read_csv(file_path+'test.csv')

In [6]:
#setting date and store as index
train_df.set_index(['Date','Store'], inplace=True)
train_df.sort_values(by=['Date','Store'], inplace=True)

test_df.set_index(['Date','Store'], inplace=True)
test_df.sort_values(by=['Date','Store'], inplace=True)

In [7]:
train_df.head(1)

,,DayOfWeek,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Year,Month,WeekOfYear,DayOfYear,CompetitionOpen,Promo2Open,Promo2running
Date,Store,,,,,,,,,,,,,,,,,
2013-01-01,85,2,8.34759,619,0,1,1,b,a,1870.0,0,2013,1,1,1,15.0,0.0,0


In [8]:
train_df.columns

Index(['DayOfWeek', 'Sales', 'Customers', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'Promo2', 'Year', 'Month', 'WeekOfYear', 'DayOfYear', 'CompetitionOpen',
       'Promo2Open', 'Promo2running'],
      dtype='object')

In [9]:
test_df.head(1)

,,DayOfWeek,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Year,Month,WeekOfYear,DayOfYear,CompetitionOpen,Promo2Open,Promo2running
Date,Store,,,,,,,,,,,,,,,,,
2015-06-15,1,1,8.615771,586,1,0,0,c,a,1270.0,0,2015,6,25,166,81.0,0.0,0


## Test-Train-Split

In [10]:
# dividing train df into train_x and train_y 
X_train = train_df.drop('Sales', axis=1)
y_train = train_df[['Sales']]

#dividing test df into test_x and test_y
X_test = test_df.drop("Sales", axis=1)
y_test = test_df[['Sales']]

One Hot Encoding of categorical columns to conver them into numerical columns

In [11]:
# making a list of categorical columns
Categorical_columns = ['DayOfWeek','StoreType','Assortment']

In [12]:
# assigning one hot encoder
enc = OneHotEncoder(sparse=False)

In [13]:
enc.fit(X_train[Categorical_columns])

OneHotEncoder(sparse=False)

In [14]:
# getting a new list of encoded columns from Categorical_columns in train data
encoded_cols = enc.get_feature_names(Categorical_columns).tolist()

In [15]:
encoded_cols

['DayOfWeek_1',
 'DayOfWeek_2',
 'DayOfWeek_3',
 'DayOfWeek_4',
 'DayOfWeek_5',
 'DayOfWeek_6',
 'DayOfWeek_7',
 'StoreType_a',
 'StoreType_b',
 'StoreType_c',
 'StoreType_d',
 'Assortment_a',
 'Assortment_b',
 'Assortment_c']

In [16]:
#performing transformation on training data and creating new features for encoded columns
X_train[encoded_cols] = enc.transform(X_train[Categorical_columns])

In [17]:
#dropping old categorical columns from training data
X_train.drop(Categorical_columns, axis=1, inplace=True)

In [18]:
#performing transformation on testing data and creating new features for encoded columns
X_test[encoded_cols] = enc.transform(X_test[Categorical_columns])

In [19]:
#dropping old categorical columns from testing data
X_test.drop(Categorical_columns, axis=1, inplace=True)

In [20]:
X_train.columns

Index(['Customers', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'CompetitionDistance', 'Promo2', 'Year', 'Month', 'WeekOfYear',
       'DayOfYear', 'CompetitionOpen', 'Promo2Open', 'Promo2running',
       'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3', 'DayOfWeek_4',
       'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7', 'StoreType_a',
       'StoreType_b', 'StoreType_c', 'StoreType_d', 'Assortment_a',
       'Assortment_b', 'Assortment_c'],
      dtype='object')

### **Time for transformations in our data**

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train[list(X_train.columns)] = scaler.fit_transform(X_train[list(X_train.columns)])
X_test[list(X_test.columns)] = scaler.fit_transform(X_test[list(X_test.columns)])

scaler = StandardScaler()
y_train[list(y_train.columns)] = scaler.fit_transform(y_train[list(y_train.columns)])
y_test[list(y_test.columns)] = scaler.transform(y_test[list(y_train.columns)])

## **Model Selection**

Going through the assumptions of linear models, we can confidently conclude that we can go for them. Given Our Data has a lot of genuine multicollinearity and also some columns have way too many outliers than others. 

Linear models like Linear regression and Logistic ones can't be used for our purpose, so we will move ahead with decision trees and randomforests, but, before that just to check.

In [22]:
lrreg = LinearRegression()

In [23]:
lrreg.fit(X_train,y_train)

LinearRegression()

In [24]:
y_train_pred = lrreg.predict(X_train)

In [25]:
y_test_pred = lrreg.predict(X_test)

In [26]:
# importing evaluation matrices
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score



In [27]:
print(f'r2_score for training data is {r2_score(y_train_pred,y_train)}')
print(f'r2_score for training data is {r2_score(y_test_pred,y_test)}')

r2_score for training data is 0.6675621318983829
r2_score for training data is -3.831468475823385e-11


As expected high bias and high variance.

### **Model 1(Baseline): DecisionTree**

In [28]:
# import the regressor
from sklearn.tree import DecisionTreeRegressor 

In [29]:
# assigning a variable
dtree = DecisionTreeRegressor(random_state=68)

In [30]:
dtree.fit(X_train,y_train)

DecisionTreeRegressor(random_state=68)

In [31]:
# predicting Y_train
y_pred_train = dtree.predict(X_train)

In [32]:
# predicting Y_train
y_pred_test = dtree.predict(X_test)

In [33]:
# printing evaluation matrices for our model
print(f'r2_score for training data is {r2_score(y_pred_train,y_train)}')
print(f'r2_score for testing data is {r2_score(y_pred_test,y_test)}')

print(f'Adjusted r2_score for training data is {round(1 - (1-r2_score(y_train, y_pred_train)) * (len(y_train)-1)/(len(y_train)-X_train.shape[1]-1),6)}')
print(f'Adjusted r2_score for testing data is {round(1 - (1-r2_score(y_test, y_pred_test)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1),6)}')

print(f'Mean Absolute Error for training data is {mean_absolute_error(y_pred_train,y_train)}')
print(f'Mean Absolute Error for testing data is {mean_absolute_error(y_pred_test,y_test)}')

print(f'Mean Squared Error for training data is {mean_squared_error(y_pred_train,y_train)}')
print(f'Mean Squared Error for testing data is {mean_squared_error(y_pred_test,y_test)}')

r2_score for training data is 0.9999987542867533
r2_score for testing data is 0.9132782268502603
Adjusted r2_score for training data is 0.999999
Adjusted r2_score for testing data is 0.911298
Mean Absolute Error for training data is 4.189885590080446e-06
Mean Absolute Error for testing data is 0.20540436100243337
Mean Squared Error for training data is 1.2457116948123794e-06
Mean Squared Error for testing data is 0.08350558487359472


As it is famously assumed, Decision trees are indeed prone to overfitting, with r2_score of 1 on traing data, it has completely overfitted the data, while for test data, it's accuracy is around 93%.

To ease this problem of overfitting we will use Random Forest to improve our model accuracy.

## **XGBoost**

In [47]:
# importing XGBoost
import xgboost as xgb

In [68]:
# assigning variable for it
xgb_reg = xgb.XGBRegressor(random_state=68)

In [69]:
# importing GridSearch CV 
from sklearn.model_selection import GridSearchCV

In [70]:
# assigning grid
param_grid = {"max_depth":[4, 5],
              "n_estimators":[100,150,None],
              "learning_rate":[0.01, 0.015]}

In [73]:
#assigning variable to GridSearchCV
search = GridSearchCV(xgb_reg, param_grid, cv=3)

In [74]:
# fitting gridcv to train data
search.fit(X_train, y_train)

[15:14:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:15:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:17:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:18:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:19:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:21:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:23:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:24:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

GridSearchCV(cv=3, estimator=XGBRegressor(random_state=68),
             param_grid={'learning_rate': [0.01, 0.015], 'max_depth': [4, 5],
                         'n_estimators': [100, 150, None]})

In [61]:
# getting best param
search.best_params_

{'learning_rate': 0.015, 'max_depth': 5, 'n_estimators': 100}

In [62]:
# storing best parameters to a variable
param = search.best_params_

In [63]:
# assigning variable with best parameters
xgb_reg = xgb.XGBRFRegressor(learning_rate = param['learning_rate'], n_estimators = param['n_estimators'], max_depth = param['max_depth'], random_state=68)

In [64]:
# fiting regressor on training data
xgb_reg.fit(X_train, y_train)

[15:07:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRFRegressor(learning_rate=0.015, max_depth=5, random_state=68)

In [65]:
# predicting y train
y_train_pred_xgb = xgb_reg.predict(X_train)

In [66]:
# predicting Y_test

y_test_pred_xgb = xgb_reg.predict(X_test)

In [67]:
# printing accuracy 
# printing evaluation matrices for our model
print(f'r2_score for training data is {r2_score(y_train_pred_xgb,y_train)}')
print(f'r2_score for testing data is {r2_score(y_test_pred_xgb,y_test)}')

print(f'Adjusted r2_score for training data is {round(1 - (1-r2_score(y_train, y_train_pred_xgb)) * (len(y_train)-1)/(len(y_train)-X_train.shape[1]-1),6)}')
print(f'Adjusted r2_score for testing data is {round(1 - (1-r2_score(y_test, y_test_pred_xgb)) * (len(y_test)-1)/(len(y_test)-X_test.shape[1]-1),6)}')

print(f'Mean Absolute Error for training data is {mean_absolute_error(y_train_pred_xgb,y_train)}')
print(f'Mean Absolute Error for testing data is {mean_absolute_error(y_test_pred_xgb,y_test)}')

print(f'Mean Squared Error for training data is {mean_squared_error(y_train_pred_xgb,y_train)}')
print(f'Mean Squared Error for testing data is {mean_squared_error(y_test_pred_xgb,y_test)}')

r2_score for training data is -7122.448857758814
r2_score for testing data is -6676.264862523788
Adjusted r2_score for training data is -0.21888
Adjusted r2_score for testing data is -0.158975
Mean Absolute Error for training data is 0.8700401842358845
Mean Absolute Error for testing data is 0.821165681601727
Mean Squared Error for training data is 1.2188389861989148
Mean Squared Error for testing data is 1.0910786415958105
